`requirements/runtime.txt`

In [11]:
import jsonpickle, os
from datetime import datetime, UTC
from dotenv import load_dotenv

from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import QiskitRuntimeService

In [12]:
load_dotenv()

service = QiskitRuntimeService.save_account(
    token=os.getenv("QISKIT_IBM_TOKEN"), 
    instance=os.getenv("QISKIT_IBM_INSTANCE"),
    overwrite=True
)

In [13]:
backends = ["ibm_fez", "ibm_marrakesh", "ibm_torino"]
service = QiskitRuntimeService()

for backend_name in backends:
    backend = service.backend(backend_name)
    noise_model = NoiseModel.from_backend(backend)

    payload = {
        "noise_model": noise_model.to_dict(),
        "coupling_map": backend.configuration().coupling_map,
    }

    ts = datetime.now(UTC).strftime("%Y%m%d_%H%M%S")    
    backend_dir = os.path.join("calibrations", backend_name)
    os.makedirs(backend_dir, exist_ok=True)

    path = os.path.join(backend_dir, f"{ts}.json")
    with open(path, "w") as f:
        f.write(jsonpickle.encode(payload, indent=2))
